In [2]:
import numpy as np
import pandas as pd
import spacy
import time
import re

import requests
import bs4
import json

In [3]:
# get urls of articles from ny times article search api request to variable "url_list"
response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=apple&fq=news_desk:Business&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
docs = response['response']['docs']
url_list = []
for item in docs:
    url_list.append(item['web_url'])
article_list = []
url_list


['https://www.nytimes.com/2020/12/16/technology/facebook-takes-the-gloves-off-in-feud-with-apple.html',
 'https://www.nytimes.com/2020/12/13/business/media/apple-gawker-tim-cook.html',
 'https://www.nytimes.com/2020/12/01/technology/amazon-apple-chips-intel-arm.html',
 'https://www.nytimes.com/2020/12/17/business/dealbook/tech-apple-facebook-fight.html',
 'https://www.nytimes.com/2020/12/17/technology/google-antitrust-monopoly.html',
 'https://www.nytimes.com/2020/12/15/technology/big-tech-regulation-europe.html',
 'https://www.nytimes.com/2020/12/14/technology/big-tech-lobbying-europe.html',
 'https://www.nytimes.com/2020/12/09/technology/personaltech/amazon-halo-review.html',
 'https://www.nytimes.com/2020/11/18/technology/apple-app-store-fee.html',
 'https://www.nytimes.com/2020/11/10/technology/apple-chips-intel.html']

In [41]:
# retrieve title, abstract, and text of each article to varialbe "text"
text = []
title = []
abstract = []
for url in url_list:
    time.sleep(0.5)
    headers = {'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}
    article = requests.get(url, headers = headers)
    soup = bs4.BeautifulSoup(article.content, 'html.parser')
    article_text_p = soup.find_all('p', attrs={'class': 'css-axufdj evys1bk0'})
    abstract_text_p = soup.find('p', attrs={'class': 'css-w6ymp8 e1wiw3jv0'})
    title_text_h1 = soup.find('h1', attrs={'data-test-id': 'headline'})
    temp = []
    title.append(title_text_h1.text)
    abstract.append(abstract_text_p.text)
    
    for item in article_text_p:
        temp.append(item.text)
    space = ' '
    article_text = space.join(temp)
    text.append(article_text)

In [47]:
tokenized_by_sentence = []
for num in range(len(text)):
   text_token = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!|;|”)\s", text[num])
   text_token.insert(0, abstract[num])
   text_token.insert(0, title[num])
   tokenized_by_sentence.append(text_token)

['Facebook Takes the Gloves Off in Feud With Apple',
 'The social network said it opposed changes that Apple was making to the tracking of apps and would provide information for an antitrust complaint against the iPhone maker.',
 'For years, signs of discord have brewed between Facebook and Apple.',
 'Their chief executives, Apple’s Tim Cook and Facebook’s Mark Zuckerberg, have periodically taken thinly veiled shots at each other.',
 '“If they’re making money mainly by collecting gobs of personal data, I think you have a right to be worried,”',
 'Mr. Cook said of companies like Facebook in 2014.',
 'In turn, Mr. Zuckerberg has retorted: “You think because you’re paying Apple that you’re somehow in alignment with them?',
 'If you were in alignment with them, then they’d make their products a lot cheaper.”',
 'But now Apple is making changes that threaten Facebook’s business — and the fight has intensified.',
 'Early next year, Apple plans to start requiring iPhone owners to explicitly c

In [7]:
# tokenize to sentence (done)
# remove mediocre words from each sentence
# take care of ing, ed, en words
# run ml